Kornpob Bhirombhakdi, kbhirombhakdi@stsci.edu

# Description
This file contains functions that can be useful.

In [1]:
import numpy as np

def read_ds9region(ds9regfile):
    """
    Assume ds9regfile in the format as ds9, and coordinate system as image
    """
    out = {}
    f = open(ds9regfile,'r')
    for i,ii in enumerate(f.readlines()):
        if i < 3:
            continue
        x,y,_ = np.array(ii.split('(')[1].split(')')[0].split(',')).astype(float)
        z = ii.split('{')[1].split('}')[0]
        out[z] = (x,y)
    return out


In [2]:
from hstgrism.container import Container
from hstgrism.computetracenwavelength import ComputeTraceNWavelength
from hstgrism.computexyref import ComputeXYREF
from hstgrism.wfc3irg102 import WFC3IRG102
from hstgrism.cutout2d import Cutout2D
from hstgrism.flatfield import FlatField
from hstgrism.show_overview import show_overview
import pandas as pd
import os

def generate_flatfield_flt(saveprefix,savefolder,plotformat,overwrite,
                           xyd,confile,beam,gfile,dfile,
                           flatfile,
                           params_overview
                          ):
    """
    Use hstgrism to make flatfielded grism flt.
    """
    ##########
    # Make Container
    ##########
    container_params = {'saveprefix':saveprefix,
                        'savefolder':savefolder,
                        'plotformat':plotformat,
                        'overwrite':overwrite
                       }
    containerobj = Container(**container_params)
    ##########
    # Compute xyref
    ##########
    xyref_params = {'xyd':xyd,
                    'xydiff':'default',
                    'xyoff':'default',
                    'confile':confile,
                    'beam':beam,
                    'gfile':gfile,
                    'dfile':dfile,
                    'container':containerobj
                   }
    xyrefobj = ComputeXYREF(**xyref_params)
    xyrefobj.compute()
    xyrefobj.save()
    ##########
    # Map trace and wavelength
    ##########
    xyref = xyrefobj.data['xyref']
    grism_params = {'confile':confile,
                    'beam':beam,
                    'xyref':xyref,
                    'container':containerobj
                   }
    grismobj = WFC3IRG102(**grism_params)
    grismobj.compute()
    grismobj.save()
    ##########
    # show_overview
    ##########
    show_overview(gfile,dfile,xyd,xyref,grismobj.output['xh'],grismobj.output['yh'],grismobj.output['ww'],
                  objname=containerobj.data['saveprefix'],
                  save=True,
                  container=containerobj,
                  params=params_overview
                 )    
    ##########
    # Make cutout
    ##########
    tfile = './{0}/{1}_trace.csv'.format(containerobj.data['savefolder'],containerobj.data['saveprefix'])
    tdata = pd.read_csv(tfile)
    xg = tdata.xh + tdata.xyref[0]
    yg = tdata.yh + tdata.xyref[1]
    halfdy = 20
    tbox = {'xg':xg,'yg':yg,'halfdyup':halfdy,'halfdylow':halfdy}
    bbox={'padxleft': 15, 'padxright': 15, 'padyup': 15, 'padylow': 15}
    image = fits.open(gfile[0])[gfile[1]].data
    do_mask = True
    cutoutobj = Cutout2D(tbox,bbox,image,containerobj)
    cutoutobj.compute(do_mask)
    cutoutobj.save()
    ##########
    # Make flatfield
    ##########
    tfile = './{0}/{1}_trace.csv'.format(containerobj.data['savefolder'],containerobj.data['saveprefix'])
    tdata = pd.read_csv(tfile)
    bbcorner = './{0}/{1}_bbcorner.csv'.format(containerobj.data['savefolder'],containerobj.data['saveprefix'])
    bbdata = pd.read_csv(bbcorner)
    flatobj = FlatField(container=containerobj,flatfile=flatfile,xyref=tdata.xyref,dldp=tdata.dldp)
    flatobj.compute()
    flatobj.save(do_cutout=False)
    ##########
    # Make flatfielded flt
    ##########
    string = './{0}/{1}'.format(containerobj.data['savefolder'],gfile[0].split('/')[-1])
    os.system('cp {0} {1}'.format(gfile[0],string))
    gfile_new = (string,1)
    ffile = ('./{0}/{1}_fullflat.fits'.format(containerobj.data['savefolder'],containerobj.data['saveprefix']),1)
    tmp = fits.open(gfile_new[0])
    tmpdata = tmp[gfile_new[1]].data.copy()
    flatdata = fits.open(ffile[0])[ffile[1]].data.copy()
    m = np.isfinite(flatdata)
    tmpdata[m] = tmpdata[m] / flatdata[m]
    tmp[gfile_new[1]].data = tmpdata.copy()
    tmp.writeto(gfile_new[0],overwrite=True)
    print('Update flatfielded {0}'.format(gfile_new[0]))


In [3]:
from astropy.io import fits

def bkgsub(gfile,bfile,container,savesuffix = 'sub'):
    """
    Perform bacgkground subtraction and save as ./savefolder/saveprefix_savesuffix.fits
    - savesuffix = 'sub' by default
    - output = gfile - bfile
    """
    savefolder = container.data['savefolder']
    saveprefix = container.data['saveprefix']
    gdata = fits.open(gfile[0])[gfile[1]].data
    bdata = fits.open(bfile[0])[bfile[1]].data
    sdata = gdata - bdata
    phdu = fits.PrimaryHDU()
    ihdu = fits.ImageHDU()
    hdul = fits.HDUList([phdu,ihdu])
    hdul[1].data = sdata
    string = './{0}/{1}_{2}.fits'.format(savefolder,saveprefix,savesuffix)
    hdul.writeto(string,overwrite=True)
    print('Save {0}'.format(string))
    

In [4]:
from astropy.io import fits
import matplotlib.pyplot as plt

def show_dither(flist):
    """
    Plot dither pattern for diagnosis
    """
    fig = plt.figure()
    ax1 = fig.add_subplot(1,2,1)
    ax1.set_title('dither pattern',fontsize=12)
    ax2 = fig.add_subplot(1,2,2)
    ax2.set_title('reduced dither pattern',fontsize=12)
    for i,ii in enumerate(flist):
        scale = fits.open(ii)[1].header['IDCSCALE']
        px,py = fits.open(ii)[0].header['POSTARG1']/scale,fits.open(ii)[0].header['POSTARG2']/scale
        ax1.plot(px,py,'s')
        ax1.annotate(i,(px,py),
                     textcoords='offset points',
                     xytext=(0,10),
                     ha='center',
                     fontsize=12,
                     rotation=0.
                    )
        x,y = np.mod(px,1.),np.mod(py,1.)
        ax2.plot(x,y,'s')
        ax2.annotate(i,(x,y),
                     textcoords='offset points',
                     xytext=(0,10),
                     ha='center',
                     fontsize=12,
                     rotation=0.
                    )
    fig.tight_layout()
        